In [98]:
import torch

from packages import *

from BranchAndBound import BranchAndBound
from NeuralNetwork import NeuralNetwork

In [99]:
# fileName = "Test3-5-3.pth"
# pathToStateDictionary = "Networks/" + fileName
# network = NeuralNetwork(pathToStateDictionary)
# dim = network.Linear[0].weight.shape[1]

# outputDim = network.Linear[-1].weight.shape[0]
# device = torch.device("cpu")
# network.to(device)

# W = []
# bias = []
# print(network.Linear)
# for i in range(len(network.Linear)):
#     if i%2:
#         continue
#     W.append(torch.Tensor(network.Linear[i].weight))
#     bias.append(torch.Tensor(network.Linear[i].bias))


# W, bias

In [100]:
# A = torch.zeros((2*dim, 2*dim))
# A[:dim, dim:] = torch.eye(dim)
# A

In [101]:
# g = 9.8

# B = torch.zeros((2*dim, dim))
# B[dim:] = torch.eye(dim)
# B[3,0] = g
# B[4,1] = -g
# B

In [102]:
# class NeuralNetworkforReachSDP(nn.Module):
#     def __init__(self, path, A, B):
#         self.statedim = 6
#         self.controlstate = 3

#         super().__init__()
#         stateDictionary = torch.load(path, map_location=torch.device("cpu"))
#         layers = []
#         for keyEntry in stateDictionary:
#             if "weight" in keyEntry:
#                 layers.append(nn.Linear(stateDictionary[keyEntry].shape[1] + self.statedim,
#                                              stateDictionary[keyEntry].shape[0] + self.statedim))
#                 layers.append(nn.ReLU())
        
#         layers.append(nn.Linear(self.statedim + self.controlstate, self.statedim))
        
#         self.Linear = nn.Sequential(
#             *layers
#         )

#         print(self.Linear[0].weight)
#         for i in range(len(network.Linear)):
#             if i%2:
#                 continue
#             self.Linear[i].weight[]
        
#         # print(self.state_dict)
#         # print(self.Linear[0].weight)

#     def load(self, path):
#         stateDict = torch.load(path, map_location=torch.device("cpu"))
#         self.load_state_dict(stateDict)

#     # @TODO
#     # def train(self):

#     def forward(self, x):
#         return self.Linear(x)

In [103]:
# network = NeuralNetworkforReachSDP(pathToStateDictionary, A, B)

### Approach 2

In [104]:
import scipy.io

In [105]:
def loadData(fileLocation="./Networks/nnmpc_nets_di_1.mat"):
    nn = scipy.io.loadmat(fileLocation)
    if 'net' in nn.keys():
        net = nn['net']
        bs = [net['biases'][0, 0][0, 0], net['biases'][0, 0][0, 1], net['biases'][0, 0][0, 2]]
        Ws = [net['weights'][0, 0][0, 0], net['weights'][0, 0][0, 1], net['weights'][0, 0][0, 2]]
        A = nn['AMatrix']
        B = nn['BMatrix']
        lu = 10 * (np.random.rand(2, 2) - 0.5)
    else:
        bs = [nn['biases'][0, 0], nn['biases'][0, 1], nn['biases'][0, 2]]
        Ws = [nn['weights'][0, 0], nn['weights'][0, 1], nn['weights'][0, 2]]
        A = np.array([[1, 1], [0, 1]])
        B = np.array([[0.5], [1]])
        lu = np.array([[1, 2],
                   [1.5, 2.5]])
    uDimension = Ws[-1].shape[0]
    # uDimension = 1  # the code is not designed for other scenarios
    uBounds = np.array([[-1, 1]])
    if uBounds.shape[0] != uDimension:
        raise "Error in uBounds"

    Ws.append(-np.eye(uDimension))
    Ws.append(-np.eye(uDimension))
    bs[-1] -= uBounds[:, 0:1]
    bs.append(uBounds[:, 1:2] - uBounds[:, 0:1])
    bs.append(uBounds[:, 1:2])

    dimensionList = [w.shape for w in Ws]
    return A, B, Ws, bs, lu, dimensionList


In [106]:
A, B, Ws, bs, lu, dimensionList = loadData()
print(dimensionList)

[(15, 2), (10, 15), (1, 10), (1, 1), (1, 1)]


In [107]:
class NeuralNetworktemp(nn.Module):
    def __init__(self, Ws, bs, dimensionList):
        super().__init__()
        layers = []
        for i in range(len(dimensionList)):
            netSize = dimensionList[i]

            layers.append(nn.Linear(netSize[1], netSize[0]))
            layers[-1].weight = torch.nn.parameter.Parameter(torch.from_numpy(Ws[i]).float())
            layers[-1].bias = torch.nn.parameter.Parameter(torch.from_numpy(bs[i][:, 0]).float())
            layers.append(nn.ReLU())

        layers.pop()
        self.Linear = nn.Sequential(
            *layers
        )

        print(self)

    def forward(self, x):
        return self.Linear(x)


In [109]:
nnet = NeuralNetworktemp(Ws, bs, dimensionList)
torch.save(nnet.state_dict(), './doubleIntegrator.pth')

NeuralNetworktemp(
  (Linear): Sequential(
    (0): Linear(in_features=2, out_features=15, bias=True)
    (1): ReLU()
    (2): Linear(in_features=15, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=1, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1, out_features=1, bias=True)
    (7): ReLU()
    (8): Linear(in_features=1, out_features=1, bias=True)
  )
)


In [96]:
print(nnet.Linear[0].bias)

Parameter containing:
tensor([-0.2750,  0.0204,  0.1331, -0.2491,  0.0058,  0.4093,  0.0043, -0.1288,
         0.1211,  0.0946,  0.0424, -0.1920,  0.4430,  0.1756, -0.0209],
       requires_grad=True)
